In [91]:
from ctgan import CTGAN
from ctgan import load_demo
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from xgboost import XGBClassifier
from xgboost import plot_importance
from catboost import CatBoostClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

In [92]:
def pos_count_great(tmp_y_pred, tmp_y_pred2, y_test, y_pred):
    num_count = []
    tp = []
    fn = []
    tp_balance_num = 0
    tp_imbalance_num = 0
    tp_same_num = 0
    fn_balance_num = 0
    fn_imbalance_num = 0
    fn_same_num = 0
    for index, row in enumerate(y_test):
        if y_test[index]==1:
            if y_pred[index]==1:
                if (tmp_y_pred[index] > tmp_y_pred2[index]) & ((tmp_y_pred[index] - tmp_y_pred2[index])>0.1):
                    tp_imbalance_num += 1
                elif (tmp_y_pred[index] < tmp_y_pred2[index]) & ((tmp_y_pred2[index] - tmp_y_pred[index] )>0.1):
                    tp_balance_num += 1
                else:
                    tp_same_num += 1
            else:
                if (tmp_y_pred[index] < tmp_y_pred2[index]) & ((tmp_y_pred2[index] - tmp_y_pred[index])>0.1):
                    fn_imbalance_num += 1
                elif (tmp_y_pred[index] > tmp_y_pred2[index]) & ((tmp_y_pred[index] - tmp_y_pred2[index])>0.1):
                    fn_balance_num += 1
                else:
                    fn_same_num += 1
        else:
            continue
    tp.append(tp_imbalance_num)
    tp.append(tp_balance_num)
    tp.append(tp_same_num)
    fn.append(fn_imbalance_num)
    fn.append(fn_balance_num)
    fn.append(fn_same_num)
    num_count.append([tp, fn])
    return num_count

In [ ]:
pd.options.display.max_columns = None
Ka = pd.read_excel("川崎症病人(訓練用 第一階段).xlsx")
fever = pd.read_excel('發燒病人(訓練用 第一階段).xlsx')
KD_val = pd.read_excel('KD_validation.xlsx')
FC_val = pd.read_excel('FC_validation.xlsx')
cmgh_20to23 = pd.read_excel('2020-23高長NewKD.xlsx')
Ka

In [ ]:
print('川崎症病人(原): ', len(Ka))
print('川崎症病人(<=5歲): ', len(Ka))
#發燒患者已只含0~6(不含6)歲
print('發燒病人: ', len(fever[fever['年齡(日)']<=1826]))
print('發燒病人: ', len(fever))


print('川崎症病人(驗證): ', len(KD_val))
print('發燒病人(驗證): ', len(FC_val))
print('川崎症病人(驗證): ', len(KD_val[KD_val['年齡(日)']<=1826]))
print('發燒病人(驗證): ', len(FC_val[FC_val['年齡(日)']<=1826]))
Ka = Ka[Ka['年齡(日)']<=1826]
fever = fever[fever['年齡(日)']<=1826]
KD_val = KD_val[KD_val['年齡(日)']<=1826]
FC_val = FC_val[FC_val['年齡(日)']<=1826]

In [95]:
cmgh_20to23['確診日期'] = pd.to_datetime(cmgh_20to23['確診日期'])
cmgh_20to23['BOD'] = pd.to_datetime(cmgh_20to23['BOD'])

cmgh_20to23['年齡(日)'] = (cmgh_20to23['確診日期']-cmgh_20to23['BOD']).dt.days

In [ ]:
print('川崎症病人(長庚驗證): ', len(cmgh_20to23[cmgh_20to23['年齡(日)']<=1826]))
# cmgh_20to23 = cmgh_20to23[cmgh_20to23['年齡(日)']<=1826]

In [97]:
#   _for_syn 為配合合成資料

#   日期轉月份
# Ka['輸入日期(月)'] = Ka['輸入日期'].dt.month
# fever['輸入日期(月)'] = fever['輸入日期'].dt.month
KD_val['輸入日期(月)'] = KD_val['輸入日期'].dt.month
FC_val['輸入日期(月)'] = FC_val['輸入日期'].dt.month
KD_val_for_syn = KD_val
FC_val_for_syn = FC_val
cmgh_20to23['輸入日期(月)'] = cmgh_20to23['確診日期'].dt.month
cmgh_20to23_for_syn = cmgh_20to23

#   one-hot
# ka_month = pd.get_dummies(Ka['輸入日期(月)'], prefix='Month')
# fever_month = pd.get_dummies(fever['輸入日期(月)'], prefix='Month')
KD_month_val = pd.get_dummies(KD_val['輸入日期(月)'], prefix='Month')
FC_month_val = pd.get_dummies(FC_val['輸入日期(月)'], prefix='Month')
val_month = pd.get_dummies(cmgh_20to23['輸入日期(月)'], prefix='Month')

#   檔案串接    raw + one-hot
# Ka = pd.concat([Ka, ka_month], axis=1)
# fever = pd.concat([fever, fever_month], axis=1)
KD_val = pd.concat([KD_val, KD_month_val], axis=1)
FC_val = pd.concat([FC_val, FC_month_val], axis=1)
cmgh_20to23 = pd.concat([cmgh_20to23, val_month], axis=1)

#   刪除多餘欄位
# Ka = Ka.drop(columns=['輸入日期(月)', '輸入日期'])  #'輸入日期(月)', 
# fever = fever.drop(columns=['輸入日期(月)', '輸入日期'])    #'輸入日期(月)', 
KD_val = KD_val.drop(columns=['輸入日期(月)', '輸入日期'])
FC_val = FC_val.drop(columns=['輸入日期(月)', '輸入日期'])
KD_val_for_syn = KD_val_for_syn.drop(columns=['輸入日期'])
FC_val_for_syn = FC_val_for_syn.drop(columns=['輸入日期'])
cmgh_20to23 = cmgh_20to23.drop(columns=['輸入日期(月)', '確診日期'])
cmgh_20to23_for_syn = cmgh_20to23_for_syn.drop(columns=['確診日期'])

In [98]:
cmgh_20to23.loc[cmgh_20to23['U/A WBC esterase']=='Positive', 'U/A WBC esterase'] = 1
cmgh_20to23.loc[cmgh_20to23['U/A WBC esterase']=='negative', 'U/A WBC esterase'] = 0
cmgh_20to23_for_syn.loc[cmgh_20to23_for_syn['U/A WBC esterase']=='Positive', 'U/A WBC esterase'] = 1
cmgh_20to23_for_syn.loc[cmgh_20to23_for_syn['U/A WBC esterase']=='negative', 'U/A WBC esterase'] = 0

#   刪除多餘欄位
cmgh_20to23 = cmgh_20to23.drop(columns=["KD","BOD","Chart No", "KD1 Age", "MCV (fL)","RDW-SD (fL)"])
cmgh_20to23_for_syn = cmgh_20to23_for_syn.drop(columns=["KD","BOD","Chart No", "KD1 Age", "MCV (fL)","RDW-SD (fL)"])

In [99]:
Ka = shuffle(Ka, random_state=30)
fever = shuffle(fever, random_state=30)
KD_val = shuffle(KD_val, random_state=30)
FC_val = shuffle(FC_val, random_state=30)
KD_val_for_syn = shuffle(KD_val_for_syn, random_state=30)
FC_val_for_syn = shuffle(FC_val_for_syn, random_state=30)

Ka['label'] = 1
fever['label'] = 0
KD_val['label'] = 1
FC_val['label'] = 0
KD_val_for_syn['label'] = 1
FC_val_for_syn['label'] = 0
cmgh_20to23['label'] = 1
cmgh_20to23_for_syn['label'] = 1

all_train_data = pd.concat([Ka, fever], ignore_index=True)
validation_data = pd.concat([KD_val, FC_val], ignore_index=True)
validation_data_for_syn = pd.concat([KD_val_for_syn, FC_val_for_syn], ignore_index=True)

In [ ]:
Ka.info()

In [101]:
x = all_train_data.values[:,:-1]
y = all_train_data.values[:,-1]
x_val = validation_data.values[:,:-1]
y_val = validation_data.values[:,-1]
x_val_for_syn = validation_data_for_syn.values[:,:-1]
y_val_for_syn = validation_data_for_syn.values[:,-1]
cmgh_20to23_x_test = cmgh_20to23.values[:,:-1]
cmgh_20to23_y_test = cmgh_20to23.values[:,-1]
cmgh_20to23_x_test_for_syn = cmgh_20to23_for_syn.values[:,:-1]
cmgh_20to23_y_test_for_syn = cmgh_20to23_for_syn.values[:,-1]

tmp_x_train, tmp_x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, stratify=y, random_state=30)
FC_test_index = [index for (index, x) in enumerate(y_test) if x==0]     #取出測試資料中FC的index
KD_test_index = [index for (index, x) in enumerate(y_test) if x==1]     #取出測試資料中KD的index
cmgh_20to23_y_test = cmgh_20to23_y_test.astype(float)
cmgh_20to23_y_test_for_syn = cmgh_20to23_y_test_for_syn.astype(float)
y_train = y_train.astype('float')
y_test = y_test.astype('float')

In [102]:
x_col = ['WBC','RBC','Hemoglobin','Hematocrit','MCH','MCHC','RDW','Platelets','Segment','Band','Lymphocyte','Monocyte','Eosinophil','Basophil','AST','ALT','CRP','UWBC','性別','年齡(日)','膿尿', '輸入日期']
y_col = ['label']
#   長庚 資料處理
#   分為合成訓練  原始訓練
df_tmp_x_train_for_syn = pd.DataFrame(tmp_x_train, columns=x_col)
df_tmp_x_test_for_syn = pd.DataFrame(tmp_x_test, columns=x_col)
df_tmp_x_train = pd.DataFrame(tmp_x_train, columns=x_col)
df_tmp_x_test = pd.DataFrame(tmp_x_test, columns=x_col)
df_tmp_y_train = pd.DataFrame(y_train, columns=y_col)

df_tmp_x_train_for_syn['輸入日期(月)'] = df_tmp_x_train_for_syn['輸入日期'].dt.month
df_tmp_x_test_for_syn['輸入日期(月)'] = df_tmp_x_test_for_syn['輸入日期'].dt.month
df_tmp_x_train['輸入日期(月)'] = df_tmp_x_train['輸入日期'].dt.month
df_tmp_x_test['輸入日期(月)'] = df_tmp_x_test['輸入日期'].dt.month

df_tmp_x_train_month = pd.get_dummies(df_tmp_x_train['輸入日期(月)'], prefix='Month')
df_tmp_x_test_month = pd.get_dummies(df_tmp_x_test['輸入日期(月)'], prefix='Month')

df_tmp_x_train = pd.concat([df_tmp_x_train, df_tmp_x_train_month], axis=1)
df_tmp_x_test = pd.concat([df_tmp_x_test, df_tmp_x_test_month], axis=1)

df_tmp_x_train_for_syn = df_tmp_x_train_for_syn.drop(columns=['輸入日期'])
df_tmp_x_test_for_syn = df_tmp_x_test_for_syn.drop(columns=['輸入日期'])
df_tmp_x_train = df_tmp_x_train.drop(columns=['輸入日期(月)', '輸入日期'])  #'輸入日期(月)', 
df_tmp_x_test = df_tmp_x_test.drop(columns=['輸入日期(月)', '輸入日期'])    #'輸入日期(月)', 

df_tmp_train_for_syn = pd.concat([df_tmp_x_train_for_syn, df_tmp_y_train], axis=1)

x_train = df_tmp_x_train.values
x_test = df_tmp_x_test.values
x_test_syn = df_tmp_x_test_for_syn.values

In [ ]:
df_tmp_x_train.info()

In [104]:
df_tmp_train_for_syn_FC = df_tmp_train_for_syn[df_tmp_train_for_syn['label']==0]
df_tmp_train_for_syn_KD = df_tmp_train_for_syn[df_tmp_train_for_syn['label']==1]

df_tmp_train_for_syn_KD = df_tmp_train_for_syn_KD.drop('label',axis=1)

float_col = ['WBC','RBC','Hemoglobin','Hematocrit','MCH','MCHC','RDW','Platelets','Segment','Band','Lymphocyte','Monocyte','Eosinophil','Basophil','AST','ALT','CRP','UWBC']
int_col = ['性別','年齡(日)','膿尿']

for i in float_col:
    df_tmp_train_for_syn_KD[i] = df_tmp_train_for_syn_KD[i].astype('float64')
for i in int_col:
    df_tmp_train_for_syn_KD[i] = df_tmp_train_for_syn_KD[i].astype('int64')

# #   建立 metadata
# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=df_tmp_train_for_syn_KD)

# #   建立模型
# synthesizer = CTGANSynthesizer(metadata)
# synthesizer.fit(df_tmp_train_for_syn_KD)

#   儲存模型
# synthesizer.save(
#     filepath='Sdv_CTGAN_0706_2.pkl'
# )

#   執行儲存的模型
# synthesizer = CTGANSynthesizer.load(
#     filepath='Sdv_CTGAN_0706_1.pkl'
# )
# synthesizer.get_parameters()

In [105]:
#   CTGAN 合成資料
# synthesizer.reset_sampling()
# print(len(df_tmp_train_for_syn_FC))
# GAN_synthetic_data = synthesizer.sample(num_rows=len(df_tmp_train_for_syn_FC)-len(df_tmp_train_for_syn_KD))
# print(len(GAN_synthetic_data))

GAN_synthetic_data = pd.read_csv("GAN_synthetic_0817_3(1_4).csv")

GAN_synthetic_data['label'] = 1
df_tmp_train_for_syn_KD['label'] = 1

#   真實資料 + 合成資料
all_train_data_GAN = pd.concat([df_tmp_train_for_syn_FC, df_tmp_train_for_syn_KD, GAN_synthetic_data], ignore_index=True)
all_train_data_GAN = shuffle(all_train_data_GAN)

x_train_GAN = all_train_data_GAN.values[:,:-1]
y_train_GAN = all_train_data_GAN.values[:,-1]
y_train_GAN = y_train_GAN.astype('float')

In [106]:
#   TVAE 合成資料
TVAE_synthetic_data = pd.read_csv("TVAE_synthetic_0707_1.csv")

TVAE_synthetic_data['label'] = 1
# df_tmp_train_for_syn_KD['label'] = 1

#   真實資料 + 合成資料
all_train_data_TVAE = pd.concat([df_tmp_train_for_syn_FC, df_tmp_train_for_syn_KD, TVAE_synthetic_data], ignore_index=True)
all_train_data_TVAE = shuffle(all_train_data_TVAE)

x_train_TVAE = all_train_data_TVAE.values[:,:-1]
y_train_TVAE = all_train_data_TVAE.values[:,-1]

In [107]:
#   驗證集整合

#   長庚raw + 高醫 + 長庚2020-22
# test_index = KD_test_index[32:133] + FC_test_index
# x_test = np.append(x_val, x_test[test_index], axis=0)
# y_test = np.append(y_val, y_test[test_index], axis=0)
# x_test = np.append(cmgh_20to23_x_test, x_test, axis=0)
# y_test = np.append(cmgh_20to23_y_test, y_test, axis=0)

# # x_test = np.append(x_val, x_test[sample(neg_index, 9000)], axis=0)
# # y_test = np.append(y_val, y_test[sample(neg_index, 9000)], axis=0)

#   使用純高醫資料驗證
x_test = x_val
y_test = y_val

#   針對訓練集中結合合成資料 驗證集整合     y_test按照先前即可
# test_index = KD_test_index[32:133] + FC_test_index
# x_test_syn = np.append(x_val_for_syn , x_test_syn[test_index], axis=0)
# x_test_syn = np.append(cmgh_20to23_x_test_for_syn, x_test_syn, axis=0)

#   高醫
x_test_syn = x_val_for_syn


In [108]:
# sm = SMOTE(random_state=42)
# x_train_balance, y_train_balance = sm.fit_resample(x_train, y_train)
# nm = NearMiss()
# x_train_balance_nm, y_train_balance_nm = nm.fit_resample(x_train, y_train)

In [109]:
#   XG + GAN
GAN_xgb = XGBClassifier()
GAN_xgb.fit(x_train_GAN, y_train_GAN)
GAN_xg_y_pred_proba = GAN_xgb.predict_proba(x_test_syn)

In [110]:
#   XG + TVAE
TVAE_xgb = XGBClassifier()
TVAE_xgb.fit(x_train_TVAE, y_train_TVAE)
TVAE_xg_y_pred_proba = TVAE_xgb.predict_proba(x_test_syn)

In [ ]:
xgb = XGBClassifier()   #scale_pos_weight=35
xgb.fit(x_train, y_train)
xg_y_pred_proba = xgb.predict_proba(x_test)
xg_thresh_list = []
xg_accuracy_score_list = []
xg_precision_score_list = []
xg_recall_score_list = []
xg_f1_score_list = []
xg_record_f2_score = []
xg_specificity_score_list = []
xg_npv_score_list = []
xg_confusion_matrix_list = []
xg_y_pred_record = []
for th in range(1,101):
    th *= 0.01

    y_pred = [0 if (x[0] >= th) else 1 for x in xg_y_pred_proba]

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    npv = tn / (fn+tn)
    xg_thresh_list.append(th)
    xg_accuracy_score_list.append(accuracy_score(y_test, y_pred) * 100)
    xg_precision_score_list.append(precision_score(y_test, y_pred) * 100)
    xg_recall_score_list.append(recall_score(y_test, y_pred) * 100)
    xg_f1_score_list.append(f1_score(y_test, y_pred) * 100)
    xg_record_f2_score.append(fbeta_score(y_test, y_pred, beta=2) * 100)
    xg_specificity_score_list.append(specificity * 100)
    xg_npv_score_list.append(npv * 100)
    xg_confusion_matrix_list.append([tn, fp, fn, tp])
    xg_y_pred_record.append(y_pred)
recall_standard_list = [80, 85, 90, 95]
compare = []
for recall_standard in recall_standard_list:
    high_recall_f1score = []
    high_recall_f2score = []
    high_recall_accuracy = []
    high_recall_recall = []
    high_recall_precision = []
    high_recall_specificity = []
    high_recall_npv = []
    high_recall_confusion_matrix = []
    high_y_pred_record = []
    for index, recall in enumerate(xg_recall_score_list):
        if recall >= recall_standard:
            high_recall_f1score.append(xg_f1_score_list[index])
            high_recall_f2score.append(xg_record_f2_score[index])
            high_recall_accuracy.append(xg_accuracy_score_list[index])
            high_recall_precision.append(xg_precision_score_list[index])
            high_recall_recall.append(xg_recall_score_list[index])
            high_recall_specificity.append(xg_specificity_score_list[index])
            high_recall_npv.append(xg_npv_score_list[index])
            high_recall_confusion_matrix.append(xg_confusion_matrix_list[index])
            high_y_pred_record.append(xg_y_pred_record[index])
    high_recall_best_f1_score_index = np.argmax(high_recall_f1score)
    print("當Recall高於", recall_standard, "% 時")
    print("且選取最高的F1-score時")
    print("Recall為為:", high_recall_recall[high_recall_best_f1_score_index], "%")
    print("Specificity為:", high_recall_specificity[high_recall_best_f1_score_index], "%")
    print("Precision為:", high_recall_precision[high_recall_best_f1_score_index], "%")
    print("NPV為:", high_recall_npv[high_recall_best_f1_score_index], "%")
    print("+LR:", (high_recall_recall[high_recall_best_f1_score_index]/100)/(1-(high_recall_specificity[high_recall_best_f1_score_index]/100)))
    print("-LR:", (1-(high_recall_recall[high_recall_best_f1_score_index]/100))/(high_recall_specificity[high_recall_best_f1_score_index]/100))
    print("F1 Score為:", high_recall_f1score[high_recall_best_f1_score_index], "%")
    print("F2 Score為:", high_recall_f2score[high_recall_best_f1_score_index], "%")
    print("Accuracy為:", high_recall_accuracy[high_recall_best_f1_score_index], "%")
    print("TN, FP, FN, TP", high_recall_confusion_matrix[high_recall_best_f1_score_index])
    # print('預測值', high_y_pred_record[high_recall_best_f1_score_index])
    print(len(high_y_pred_record[high_recall_best_f1_score_index]))
    print("============================================")
    compare.append(high_y_pred_record[high_recall_best_f1_score_index])

In [ ]:
est = [ 
    # ('gn', GaussianNB()),
    # ('mn', MultinomialNB()),
    ('ab', AdaBoostClassifier(n_estimators=59)),   
    ('xgb', XGBClassifier(scale_pos_weight=34)), 
    ('cat', CatBoostClassifier()),
    # ('rf', RandomForestClassifier()),   #n_estimators=200
    # ('dt', DecisionTreeClassifier()),   #class_weight={1: 1000}
    # ('svc', SVC(class_weight={1: 10}, probability=True)),
    # ('knn', KNeighborsClassifier(n_neighbors=100)),   
    # ('gb', GradientBoostingClassifier())
]


record_recall = []
record_specificity = []
record_precision = []
record_npv = []
record_f1_score = []
record_f2_score = []
record_accuracy = []
record_y_pred_pr = []
sequences = [1, 11, 19, 40, 73, 139]
# for i in sequences:
for i in range(30, 31):
    clf = StackingClassifier(estimators=est, final_estimator=LogisticRegression(class_weight={1:i}), cv=5)      #LogisticRegression(class_weight={1:i})
    clf.fit(x_train, y_train)
    tmp_y_pred = clf.predict_proba(x_test)
    y_pred = [0 if (x[0] >= 0.5) else 1 for x in tmp_y_pred]
    tmp_y_pred_pr = [x[1] for x in tmp_y_pred]
    record_y_pred_pr.append(tmp_y_pred_pr)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    npv = tn / (tn+fn)
    record_precision.append(precision_score(y_test, y_pred))
    record_npv.append(npv)
    record_recall.append(recall_score(y_test, y_pred))
    record_specificity.append(specificity)
    record_f1_score.append(f1_score(y_test, y_pred))
    record_f2_score.append(fbeta_score(y_test, y_pred, beta=2))
    record_accuracy.append(accuracy_score(y_test, y_pred))

In [ ]:
est2 = [     #tmp best: ada, xgb, cat 
    ('ab', AdaBoostClassifier()), 
    ('xgb', XGBClassifier()),    
    ('cat', CatBoostClassifier()),
]

# record_recall2 = []
# record_specificity2 = []
# record_precision2 = []
# record_npv2 = []
# record_f1_score2 = []
# record_f2_score2 = []
# record_accuracy2 = []
# record_y_pred2_pr = []
#配200 1900
for i in range(5, 6):
    clf2 = StackingClassifier(estimators=est2, final_estimator=LogisticRegression(class_weight={1:i}), cv=5, n_jobs=5)      #LogisticRegression(class_weight={1:i})
    clf2.fit(x_train_GAN, y_train_GAN)
    tmp_y_pred2 = clf2.predict_proba(x_test_syn)
    # y_pred2 = [0 if (x[0] >= 0.5) else 1 for x in tmp_y_pred2]
    # tmp_y_pred2_pr = [x[1] for x in tmp_y_pred2]
    # record_y_pred2_pr.append(tmp_y_pred2_pr)
    # tn, fp, fn, tp = confusion_matrix(y_test, y_pred2).ravel()
    # specificity = tn / (tn+fp)
    # npv = tn / (tn+fn)
    # record_precision2.append(precision_score(y_test, y_pred2))
    # record_npv2.append(npv)
    # record_recall2.append(recall_score(y_test, y_pred2))
    # record_specificity2.append(specificity)
    # record_f1_score2.append(f1_score(y_test, y_pred2))
    # record_f2_score2.append(fbeta_score(y_test, y_pred2, beta=2))
    # record_accuracy2.append(accuracy_score(y_test, y_pred2))


In [ ]:
y_pred_append_0 = np.append([tmp_y_pred[:,0]], [tmp_y_pred2[:,0]], axis=0)
mean_y_pred_0 = np.mean(y_pred_append_0, axis=0)
y_pred_append_1 = np.append([tmp_y_pred[:,1]], [tmp_y_pred2[:,1]], axis=0)
mean_y_pred_1 = np.mean(y_pred_append_1, axis=0)
y_pred_append = np.append([mean_y_pred_0], [mean_y_pred_1], axis=0)
soft_y_pred = [0 if (x >= 0.5) else 1 for x in y_pred_append[0]]

tn, fp, fn, tp = confusion_matrix(y_test, soft_y_pred).ravel()
specificity = tn / (tn+fp)
npv = tn / (tn+fn)
print('recall: ', recall_score(y_test, soft_y_pred))
print('specificity: ', specificity)
print('precision: ', precision_score(y_test, soft_y_pred))
print('NPV: ', npv)
print('f1_score: ', f1_score(y_test, soft_y_pred))
print('f2_score: ', fbeta_score(y_test, soft_y_pred, beta=2))
print('accuracy_score: ', accuracy_score(y_test, soft_y_pred))

In [ ]:

for i in range(1):
    print('----------------------------------------------')
    # print(sequences[i])
    print(i+30)
    print('recall: ', record_recall[i])
    print('specificity: ', record_specificity[i])
    print('precision: ', record_precision[i])
    print('NPV: ', record_npv[i])
    print('f1_score: ', record_f1_score[i])
    print('f2_score: ', record_f2_score[i])
    print('accuracy_score: ', record_accuracy[i])
    print("+LR:", (record_recall[i])/(1-(record_specificity[i])))
    print("-LR:", (1-(record_recall[i]))/(record_specificity[i]))

In [116]:
# print(sorted(list(record_y_pred_pr[0]))[:10])
# print(sorted(list(record_y_pred_pr[1]))[:10])
# print(sorted(list(record_y_pred_pr[2]))[:10])
# print(sorted(list(record_y_pred_pr[3]))[:10])

In [ ]:
#pr curve
for i in record_y_pred_pr:
    xg_y_pred_pr = [x[1] for x in xg_y_pred_proba]
    GAN_xg_y_pred_pr = [x[1] for x in GAN_xg_y_pred_proba]
    TVAE_xg_y_pred_pr = [x[1] for x in TVAE_xg_y_pred_proba]
    y_pred_append_pr = [x for x in y_pred_append[1]]
    x = np.arange(0, 1, 0.001)
    y = np.arange(1, 0, -0.001)
    fig, ax = plt.subplots()
    xg_precision, xg_recall, xg_thresholds = metrics.precision_recall_curve(y_test, xg_y_pred_pr, pos_label=1)
    plt.plot(xg_recall, xg_precision, label = "XG (val)")       #, color='m'
    # PrecisionRecallDisplay.from_predictions(y_test, xg_y_pred_pr, name="XG PR curve", ax=ax)
    GAN_xg_precision, GAN_xg_recall, GAN_xg_thresholds = metrics.precision_recall_curve(y_test, GAN_xg_y_pred_pr, pos_label=1)
    plt.plot(GAN_xg_recall, GAN_xg_precision, label = "CTGAN + XG (val)")     #, color='b'
    # PrecisionRecallDisplay.from_predictions(y_test, GAN_xg_y_pred_pr, name="CTGAN + XG PR curve", ax=ax)
    TVAE_xg_precision, TVAE_xg_recall, TVAE_xg_thresholds = metrics.precision_recall_curve(y_test, TVAE_xg_y_pred_pr, pos_label=1)
    plt.plot(TVAE_xg_recall, TVAE_xg_precision, label = "TVAE + XG (val)")     #, color='b'
    # PrecisionRecallDisplay.from_predictions(y_test, TVAE_xg_y_pred_pr, name="TVAE + XG PR curve", ax=ax)
    precision, recall, thresholds = metrics.precision_recall_curve(y_test, i, pos_label=1)
    plt.plot(recall, precision, label = "DC (val)")    #, color='y'
    # PrecisionRecallDisplay.from_predictions(y_test, i, name="ENS PR curve", ax=ax)
    CT_ENS_precision, CT_ENS_recall, CT_ENS_thresholds = metrics.precision_recall_curve(y_test, y_pred_append_pr, pos_label=1)
    plt.plot(CT_ENS_recall, CT_ENS_precision, label = "CTGAN-DC (val)")    #, color='y'
    # PrecisionRecallDisplay.from_predictions(y_test, y_pred_append_pr, name="ENS+ PR curve", ax=ax)
    # plt.plot(x, y, color='0', linestyle = "-.")

    plt.ylabel("Precision")
    plt.xlabel("Recall")
    plt.legend(loc=3)
    plt.grid(True)
    # plt.savefig("0to5_pr_curve_CTGENS(val_pure_kao).svg", format = "svg", dpi=500)
    plt.show()

In [ ]:

for i in range(len(CT_ENS_thresholds)):
    print('precision: ', CT_ENS_precision[i], ' recall: ', CT_ENS_recall[i], ' thresholds: ', CT_ENS_thresholds[i])


In [ ]:
CTENS_thresh_list = []
CTENS_accuracy_score_list = []
CTENS_precision_score_list = []
CTENS_recall_score_list = []
CTENS_f1_score_list = []
CTENS_record_f2_score = []
CTENS_specificity_score_list = []
CTENS_npv_score_list = []
CTENS_confusion_matrix_list = []
CTENS_y_pred_record = []
CTENS_tp_fn_num_count = []
for th in CT_ENS_thresholds:
    
    y_pred = [0 if (x < th) else 1 for x in y_pred_append[1]]

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    npv = tn / (fn+tn)
    CTENS_thresh_list.append(th)
    CTENS_accuracy_score_list.append(accuracy_score(y_test, y_pred) * 100)
    CTENS_precision_score_list.append(precision_score(y_test, y_pred) * 100)
    CTENS_recall_score_list.append(recall_score(y_test, y_pred) * 100)
    CTENS_f1_score_list.append(f1_score(y_test, y_pred) * 100)
    CTENS_record_f2_score.append(fbeta_score(y_test, y_pred, beta=2) * 100)
    CTENS_specificity_score_list.append(specificity * 100)
    CTENS_npv_score_list.append(npv * 100)
    CTENS_confusion_matrix_list.append([tn, fp, fn, tp])
    CTENS_y_pred_record.append(y_pred)
    CTENS_tp_fn_num_count.append(pos_count_great(tmp_y_pred[:, 1], tmp_y_pred2[:, 1], y_test, y_pred))
recall_standard_list = [80, 85, 90, 95] #
compare = []
for recall_standard in recall_standard_list:
    high_recall_f1score = []
    high_recall_f2score = []
    high_recall_accuracy = []
    high_recall_recall = []
    high_recall_precision = []
    high_recall_specificity = []
    high_recall_npv = []
    high_recall_confusion_matrix = []
    high_y_pred_record = []
    high_tp_fn_num_count = []
    for index, recall in enumerate(CTENS_recall_score_list):
        if recall >= recall_standard:
            high_recall_f1score.append(CTENS_f1_score_list[index])
            high_recall_f2score.append(CTENS_record_f2_score[index])
            high_recall_accuracy.append(CTENS_accuracy_score_list[index])
            high_recall_precision.append(CTENS_precision_score_list[index])
            high_recall_recall.append(CTENS_recall_score_list[index])
            high_recall_specificity.append(CTENS_specificity_score_list[index])
            high_recall_npv.append(CTENS_npv_score_list[index])
            high_recall_confusion_matrix.append(CTENS_confusion_matrix_list[index])
            high_y_pred_record.append(CTENS_y_pred_record[index])
            high_tp_fn_num_count.append(CTENS_tp_fn_num_count[index])
    high_recall_best_f2_score_index = np.argmax(high_recall_f2score)
    print("當Recall高於", recall_standard, "% 時")
    print("且選取最高的F2-score時")
    print("Recall為為:", high_recall_recall[high_recall_best_f2_score_index], "%")
    print("Specificity為:", high_recall_specificity[high_recall_best_f2_score_index], "%")
    print("Precision為:", high_recall_precision[high_recall_best_f2_score_index], "%")
    print("NPV為:", high_recall_npv[high_recall_best_f2_score_index], "%")
    print("+LR:", (high_recall_recall[high_recall_best_f2_score_index]/100)/(1-(high_recall_specificity[high_recall_best_f2_score_index]/100)))
    print("-LR:", (1-(high_recall_recall[high_recall_best_f2_score_index]/100))/(high_recall_specificity[high_recall_best_f2_score_index]/100))
    print("F1 Score為:", high_recall_f1score[high_recall_best_f2_score_index], "%")
    print("F2 Score為:", high_recall_f2score[high_recall_best_f2_score_index], "%")
    print("Accuracy為:", high_recall_accuracy[high_recall_best_f2_score_index], "%")
    print("TN, FP, FN, TP", high_recall_confusion_matrix[high_recall_best_f2_score_index])
    print("CTGAN-DC的TP及FN的數量:", high_tp_fn_num_count[high_recall_best_f2_score_index])
    # print('預測值', high_y_pred_record[high_recall_best_f2_score_index])
    print(len(high_y_pred_record[high_recall_best_f2_score_index]))
    print("============================================")
    compare.append(high_y_pred_record[high_recall_best_f2_score_index])

In [ ]:
# roc curve
for i in record_y_pred_pr:
    xg_y_pred_roc = [x[1] for x in xg_y_pred_proba]
    GAN_xg_y_pred_roc = [x[1] for x in GAN_xg_y_pred_proba]
    TVAE_xg_y_pred_roc = [x[1] for x in TVAE_xg_y_pred_proba]
    y_pred_append_roc = [x for x in y_pred_append[1]]
    x = np.arange(0, 1, 0.001)
    y = np.arange(0, 1, 0.001)
    fig, ax = plt.subplots()
    xg_fpr, xg_tpr, thresholds = metrics.roc_curve(y_test, xg_y_pred_roc, pos_label=1)
    plt.plot(xg_fpr, xg_tpr, label = "XG (val)")   #, color='m'
    # RocCurveDisplay.from_predictions(y_test, xg_y_pred_roc, name="XG ", ax=ax)
    GAN_xg_fpr, GAN_xg_tpr, thresholds = metrics.roc_curve(y_test, GAN_xg_y_pred_roc, pos_label=1)
    plt.plot(GAN_xg_fpr, GAN_xg_tpr, label = "CTGAN + XG (val)")     #, color='b'
    # RocCurveDisplay.from_predictions(y_test, GAN_xg_y_pred_roc, name="CTGAN + XG ", ax=ax)
    TVAE_xg_fpr, TVAE_xg_tpr, thresholds = metrics.roc_curve(y_test, TVAE_xg_y_pred_roc, pos_label=1)
    plt.plot(TVAE_xg_fpr, TVAE_xg_tpr, label = "TVAE + XG (val)")     #, color='b'
    # RocCurveDisplay.from_predictions(y_test, TVAE_xg_y_pred_roc, name="TVAE + XG ", ax=ax)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, i, pos_label=1)
    plt.plot(fpr, tpr, label = "DC (val)")        #, color='y'
    # RocCurveDisplay.from_predictions(y_test, i, name="ENS ", ax=ax)
    CT_ENS_fpr, CT_ENS_tpr, CT_ENS_thresholds = metrics.roc_curve(y_test, y_pred_append_roc, pos_label=1)
    plt.plot(CT_ENS_fpr, CT_ENS_tpr, label = "CTGAN-DC (val)")        #, color='y'
    # RocCurveDisplay.from_predictions(y_test, y_pred_append_roc, name="ENS+ ", ax=ax)
    plt.plot(x, y, color='0', linestyle = "-.")
    plt.ylabel("True Positive Rate")
    plt.xlabel("False Positive Rate")
    plt.legend(loc=0)
    plt.grid(True)
    # plt.savefig("0to5_roc_curve_CTGENS(val_pure_kao).svg", format = "svg", dpi=500)
    plt.show()